In [ ]:
# This notebook is deprecated by the manual_validation notebook

In [66]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

from lightgbm import LGBMClassifier
from itertools import chain, combinations

In [67]:
import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [68]:
fr_groundtruths =['USER_FR', 'URLa_FR', 'URLb_FR']
lr_groundtruths =['HASHTAG','URL_LR','POLITICIAN_LR','POLITICIAN_1H_LR', 'PARTY_FOLLOWER_LR']
groundtruths =  lr_groundtruths+fr_groundtruths
features = ['use', 'ht', 'rt']

In [69]:
performance = {dataset : {gt: {} for gt in groundtruths} for dataset in ['qanda']}

In [70]:
def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(1, len(ss)+1)))

In [71]:
dataset='qanda'
granularity = '_per_user'
for feature_set in all_subsets(features):

    #Check if the features exist for this dataset + granularity + feature combination
    feature_paths = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in feature_set]
    if not all([os.path.exists(fp) for fp in feature_paths]):
        continue
    #Check if the ground truth exists
    for gt in groundtruths:
        print(dataset,granularity,feature_set,gt)
        gt_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+gt+'_'+granularity+'.pk')
        if not os.path.exists(gt_path):
            print(gt_path, ": Does not exist")
            continue

        def load_vector(fp):
            with open(fp, 'rb') as rf:
                vec = pk.load(rf)
            return vec

        y_orig = load_vector(gt_path).values
        mask = y_orig >= 0
        assert any(mask)
        y = y_orig[mask]
        X_orig = np.hstack([load_vector(fp) for fp in feature_paths])
        X = X_orig[mask]

        et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)

        # validation instance
        skf = StratifiedKFold(n_splits=5, shuffle=True)

        # getting the model validation predictions
        preds = cross_val_predict(et, X, y, cv=skf, method='predict_proba')
        if gt in fr_groundtruths:
            preds = preds[:,1]
        # evaluating the model
        print(dataset,granularity,feature_set,gt)
        print('Area under the ROC Curve:', roc_auc_score(y, preds, multi_class='ovo'))

        performance[dataset][gt][feature_set] = roc_auc_score(y, preds, multi_class='ovo')

qanda _per_user ('use',) HASHTAG
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda _per_user ('use',) HASHTAG
Area under the ROC Curve: 0.8812194268501896
qanda _per_user ('use',) URL_LR
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_i

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda _per_user ('use',) POLITICIAN_LR
Area under the ROC Curve: 0.638095238095238
qanda _per_user ('use',) POLITICIAN_1H_LR


/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda _per_user ('use',) POLITICIAN_1H_LR
Area under the ROC Curve: 0.5398686424577973
qanda _per_user ('use',) PARTY_FOLLOWER_LR
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[L

...

[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500


/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500


/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


qanda _per_user ('use', 'ht', 'rt') URLb_FR
Area under the ROC Curve: 0.9280185047340319


In [74]:
performance_cv_df = pd.DataFrame({k : {'+'.join(list(k1)) : v1 for k1,v1 in v.items()} for k,v in performance['qanda'].items()})

In [77]:
performance_cv_df.to_csv(os.path.join(base_dir, 'data', '04_results', 'ablation_qanda_cv.csv'))